Grab the dataset from Kaggle

In [11]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "Heart_Disease_Prediction.csv"

df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "neurocipher/heartdisease",
  file_path
)

df.head()


,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,Presence
1,67,0,3,115,564,0,2,160,0,1.6,2,0,7,Absence
2,57,1,2,124,261,0,0,141,0,0.3,1,0,7,Presence
3,64,1,4,128,263,0,0,105,1,0.2,2,1,7,Absence
4,74,0,2,120,269,0,2,121,1,0.2,1,1,3,Absence


In [10]:
df.describe()

,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium
count,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.00000,270.000000,270.000000,270.000000
mean,54.433333,0.677778,3.174074,131.344444,249.659259,0.148148,1.022222,149.677778,0.329630,1.05000,1.585185,0.670370,4.696296
std,9.109067,0.468195,0.950090,17.861608,51.686237,0.355906,0.997891,23.165717,0.470952,1.14521,0.614390,0.943896,1.940659
min,29.000000,0.000000,1.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.00000,1.000000,0.000000,3.000000
25%,48.000000,0.000000,3.000000,120.000000,213.000000,0.000000,0.000000,133.000000,0.000000,0.00000,1.000000,0.000000,3.000000
50%,55.000000,1.000000,3.000000,130.000000,245.000000,0.000000,2.000000,153.500000,0.000000,0.80000,2.000000,0.000000,3.000000
75%,61.000000,1.000000,4.000000,140.000000,280.000000,0.000000,2.000000,166.000000,1.000000,1.60000,2.000000,1.000000,7.000000
max,77.000000,1.000000,4.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.20000,3.000000,3.000000,7.000000


In [12]:


labels_str = df["Heart Disease"]
labels_str



0      Presence
1       Absence
2      Presence
3       Absence
4       Absence
         ...   
265     Absence
266     Absence
267     Absence
268     Absence
269    Presence
Name: Heart Disease, Length: 270, dtype: str

In [13]:
labels = labels_str.map({'Absence': 0, 'Presence': 1})
labels.head()

0    1
1    0
2    1
3    0
4    0
Name: Heart Disease, dtype: int64

In [14]:
labels.unique()

array([1, 0])

This confirms `labels` is binary 

<b>Logistic Regression with One Input Feature</b>


We use the feature 'Cholesterol'.

In [8]:
features = df["Cholesterol"]
features.head()


0    322
1    564
2    261
3    263
4    269
Name: Cholesterol, dtype: int64

In [15]:
x = features
y = labels

In [16]:
from sklearn.linear_model import LogisticRegression


from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

import pandas as pd

In [19]:
standardized_data = pd.DataFrame(scaler.fit_transform(x.values.reshape(-1,1)), columns=['cholesterol'])
standardized_data.head()

,cholesterol
0,1.402212
1,6.093004
2,0.219823
3,0.258589
4,0.374890


Now we split data so some of it used used for training and the rest for testing.

In [20]:
from sklearn.model_selection import train_test_split



x_train, x_test, y_train, y_test = train_test_split(standardized_data, y, test_size=0.2, shuffle=True, random_state=42)
print("x_train shape", x_train.shape)
print("x_test shape:", x_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

x_train shape (216, 1)
x_test shape: (54, 1)
y_train shape: (216,)
y_test shape: (54,)
